In [212]:
import pandas as pd

In [3]:
import os
import numpy as np
import pydicom
import scipy.misc
import scipy

In [4]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

In [5]:
base_path = '/Users/lifesailor/Desktop/project/medical-image-synthesis-gan/data/train/'
contrast_path = 'CT(CE)/2.PG+30/'
original_path = 'CT/2.PG+30/'

In [6]:
files = os.listdir(base_path)
files.sort()
files = [file for file in files if '_' not in file and ' ' not in file]

In [7]:
original_total_path = [os.path.join(os.path.join(base_path, file), original_path) for file in files]
contrast_total_path = [os.path.join(os.path.join(base_path, file), contrast_path) for file in files]

# 1. Get File and Image

### 1. File

In [8]:
from tqdm import tqdm, tqdm_notebook, trange

In [16]:
def load_files(path):
    total_files = []
    
    # list
    path = tqdm_notebook(path)
    
    for base_path in path:    
        files = os.listdir(base_path)
        files.sort()
        
        # files
        folder_files = []
        for file in files:
            data = pydicom.dcmread(os.path.join(base_path, file))
            folder_files.append(data)
        total_files.append(folder_files)
        
    return total_files

In [17]:
original_total_files = load_files(original_total_path)
contrast_total_files = load_files(contrast_total_path)

### 1. 회사 종류

In [191]:
def get_manufaturer(file):
    file_info = file[0]
    file_info = str(file_info)
    manufacturer = file_info.split('Manufacturer')[1]
    manufacturer = manufacturer.split('\n')[0].split('LO:')[1]
    return manufacturer

In [192]:
manufacturer = []

for file in original_total_files:
    manufacturer.append(get_manufaturer(file))

In [213]:
pd.Series(manufacturer).value_counts()

 'SIEMENS'               74
 'GE MEDICAL SYSTEMS'    72
dtype: int64

### 2. 모델 종류

In [179]:
def get_model(file):
    file_info = file[0]
    file_info = str(file_info)
    model = file_info.split('Manufacturer\'s Model Name')[1]
    model = model.split('\n')[0]
    return model

In [180]:
models = []

for file in original_total_files:
    models.append(get_model(file))

In [214]:
pd.Series(models).value_counts()

           LO: 'SMS Definition AS+'       41
           LO: 'GMS Discovery 750\x00'    40
           LO: 'SMS Definition'           33
           LO: 'GMS Revolution_TC\x00'    28
           LO: 'GMS Discovery 750_CC'      4
dtype: int64

### 2. Image

In [40]:
def load_images(path):
    total_files = []
    
    # list
    path = tqdm_notebook(path)
    
    for base_path in path:    
        files = os.listdir(base_path)
        files.sort()
        
        # files
        folder_files = []
        for file in files:
            data = pydicom.dcmread(os.path.join(base_path, file))
            folder_files.append(data.pixel_array)
        total_files.append(folder_files)
        
    return total_files

In [41]:
original_total_images = load_images(original_total_path)
contrast_total_images = load_images(contrast_total_path)

In [219]:
original_total_files[0]

[(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
 (0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
 (0008, 0016) SOP Class UID                       UI: CT Image Storage
 (0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.1.4.60474.30000018070123384828100021773
 (0008, 0020) Study Date                          DA: '20180702'
 (0008, 0021) Series Date                         DA: '20180702'
 (0008, 0022) Acquisition Date                    DA: '20180702'
 (0008, 0023) Content Date                        DA: '20180702'
 (0008, 002a) Acquisition DateTime                DT: '20180702162439.260000'
 (0008, 0030) Study Time                          TM: '162320.862000'
 (0008, 0031) Series Time                         TM: '162617.258000'
 (0008, 0032) Acquisition Time                    TM: '162439.260000'
 (0008, 0033) Content Time                        TM: '162439.260000'
 (0008, 0050) Accession Number  

# 2. Imshow

- 하나의 sequence 대해서 비슷한지 확인하기

In [11]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [198]:
def f(files, sequence):
    fig = plt.figure(figsize=(15, 5))
    print(models[files:files+3])
    
    fig.add_subplot(1, 3, 1)
    plt.imshow(original_total_images[files][sequence], cmap='gray')

    fig.add_subplot(1, 3, 2)
    plt.imshow(original_total_images[files + 1][sequence], cmap='gray')

    fig.add_subplot(1, 3, 3)
    plt.imshow(original_total_images[files + 2][sequence], cmap='gray')

In [199]:
interactive_plot = interactive(f, files=range(0, len(files), 3), sequence=range(30))
interactive_plot

interactive(children=(Dropdown(description='files', options=(0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 3…

In [205]:
def f2(files, sequence):
    fig = plt.figure(figsize=(15, 5))
    print(models[files:files+3])
    
    fig.add_subplot(1, 3, 1)
    plt.imshow(contrast_total_images[files][sequence], cmap='gray')

    fig.add_subplot(1, 3, 2)
    plt.imshow(contrast_total_images[files + 1][sequence], cmap='gray')

    fig.add_subplot(1, 3, 3)
    plt.imshow(contrast_total_images[files + 2][sequence], cmap='gray')

In [206]:
interactive_plot = interactive(f2, files=range(0, len(files), 3), sequence=range(30))
interactive_plot

interactive(children=(Dropdown(description='files', options=(0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 3…

# 3. Training

### 데이터 수집

1. 하나의 장비의 데이터를 많이 모을수록 좋습니다.
    - 적어도 같은 회사이어야 합니다.
    - 쉬운 문제부터 풀고 어려운 문제를 풉니다.
    - 하나의 장비에서 하는 것이 더 쉽습니다.

### 알고리즘에 사용할 데이터 관점

1. 회사
    - 문제는 데이터 양이 줄어든다
         1) 나누어서 실험하기 2) multi-task 등으로 해결해야 한다.
    - 둘 다 실험해야 합니다.
2. Sequence 
    - sequence를 구분해서 넣는다.
        1) 추가로 sequence 정보를 넣는다 2) 중요 sequence만 뽑는다

### 논의 사항

1. dicom 파일에 많은 정보가 들어가 있다.
    - 이것을 같이 쓸 수 있는 방법은 없는가.
    - 우선 이미지만 가지고 한다.

### 단계
  
1. Segmentation
    - Skull을 딴다.
    - Inner을 딴다. (crop)
    

2. Augmentation: Input
    - Rotation
    - Size
    

3. Preprocessing
    - Clahe - input: 두드러지게 한다.
    - Binarization - output: 이진화 한다.

### 실험

실험 6가지 실험

- 데이터 3가지: GE / SIMENS / GE + SIMENS
- 전처리 2가지: INPUT - SEGMENTATION / CROP
- 모델: PIX2PIXHD 통일

주제

1. Sequence
2. 3D